### Installing Necessary Libraries

Here we install all the necessary libraries required for Setting Up Vision Transformer with Relative Positional Encoding. We rely on an existing implementation of image RPE [here](https://github.com/microsoft/Cream/tree/main/iRPE/DETR-with-iRPE).

In [ ]:
# iRPE
!git clone https://github.com/microsoft/Cream.git
%cd Cream/iRPE/DETR-with-iRPE
!pip install -r ./requirements.txt

# iRPE for CUDA
%cd rpe_ops
!python setup.py install --user
%cd ..

### Import Libraries

In [6]:
# General
from pathlib import Path
from typing import List, Any
from io import BytesIO
import os
import zipfile

# Data Handling
import torch.utils.data
from pycocotools import mask as coco_mask
from PIL import Image

# Model Building
import torch
import torchvision

### Load Data

Here we will load the dataset for training the Object Detection Model, with RPE.

In [7]:
# Folder to store the data
!mkdir coco_data
%cd coco_data
# Train
!wget http://images.cocodataset.org/zips/train2017.zip
# Validation
!wget http://images.cocodataset.org/zips/val2017.zip
# Annotations
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
# Unzip Annotations
!unzip annotations_trainval2017.zip -d .
!rm annotations_trainval2017.zip
%cd ..
# Stores Output
!mkdir output

/content/Cream/iRPE/DETR-with-iRPE/coco_data
--2023-03-08 04:10:36--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.0.1, 52.217.167.113, 52.216.145.251, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.0.1|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘train2017.zip’

train2017.zip       100%[===================>]  18.01G  43.8MB/s    in 4m 28s  

2023-03-08 04:15:04 (68.8 MB/s) - ‘train2017.zip’ saved [19336861798/19336861798]

--2023-03-08 04:15:04--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.235.89, 54.231.235.9, 3.5.3.185, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.235.89|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip’

val2017.zip      

In [ ]:
!python -m torch.distributed.launch --nproc_per_node=1 --use_env main.py --lr_drop 100 --epochs 150 --coco_path ./coco_data --enc_rpe2d rpe-2.0-product-ctx-1-k --output_dir ./output